In [10]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import ace_tools_open as tools
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

In [2]:
df = pd.read_csv('data_football_ratings.csv')

# I. Statistiques descriptives

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50652 entries, 0 to 50651
Data columns (total 63 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   competition             50652 non-null  object 
 1   date                    50652 non-null  object 
 2   match                   50652 non-null  object 
 3   team                    50652 non-null  object 
 4   pos                     50652 non-null  object 
 5   pos_role                50652 non-null  object 
 6   player                  50652 non-null  object 
 7   rater                   50652 non-null  object 
 8   is_human                50652 non-null  int64  
 9   original_rating         50652 non-null  float64
 10  goals                   50652 non-null  int64  
 11  assists                 50652 non-null  int64  
 12  shots_ontarget          50652 non-null  int64  
 13  shots_offtarget         50652 non-null  int64  
 14  shotsblocked            50652 non-null

In [7]:
df.describe()

,is_human,original_rating,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,drib_success,drib_unsuccess,...,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,win,lost,is_home_team,minutesPlayed,game_duration
count,50652.000000,50652.000000,50652.000000,50652.000000,50652.000000,50652.000000,50652.000000,50652.000000,50652.000000,50652.000000,...,50652.000000,50652.000000,50652.000000,50652.000000,50652.000000,50652.000000,50652.000000,50652.000000,50652.000000,50652.000000
mean,0.530996,5.687847,0.099601,0.061893,0.299041,0.557135,0.214049,0.568053,0.524027,0.347686,...,0.169482,0.661259,0.317616,0.813626,0.011549,0.369008,0.373806,0.406460,79.371180,90.456053
std,0.499043,1.673356,0.335712,0.257212,0.631347,0.878827,0.505860,0.935402,0.955628,0.710788,...,0.086042,0.102062,0.145349,0.683888,0.035372,0.482541,0.483818,0.491177,21.425548,3.670678
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.008130,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,90.000000
25%,0.000000,4.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.106612,0.593939,0.216667,0.202468,0.000000,0.000000,0.000000,0.000000,78.000000,90.000000
50%,1.000000,6.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.163850,0.670635,0.321839,0.744910,0.000000,0.000000,0.000000,0.000000,90.000000,90.000000
75%,1.000000,7.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,0.225616,0.725926,0.420000,1.264387,0.000000,1.000000,1.000000,1.000000,90.000000,90.000000
max,1.000000,10.000000,4.000000,3.000000,7.000000,7.000000,4.000000,8.000000,10.000000,7.000000,...,0.525179,0.933333,0.810811,6.074042,1.000000,1.000000,1.000000,1.000000,120.000000,120.000000


In [3]:
categorical_summary = df.describe(include=["object"])
tools.display_dataframe_to_user(name="Descriptive Statistics - Numerical Data", dataframe=df.describe())
tools.display_dataframe_to_user(name="Descriptive Statistics - Categorical Data", dataframe=categorical_summary)

Descriptive Statistics - Numerical Data


Descriptive Statistics - Categorical Data


In [4]:
df_grouped = df.groupby(['player', 'match'], as_index=False).agg({
    'original_rating': 'mean',
    'team': 'first',
    'pos': 'first',
    'pos_role': 'first',
    'is_human': 'mean',
    'goals': 'first',
    'assists': 'first',
    'shots_ontarget': 'first',
    'shots_offtarget': 'first',
    'shotsblocked': 'first',
    'chances2score': 'first',
    'drib_success': 'first',
    'drib_unsuccess': 'first',
    'keypasses': 'first',
    'touches': 'first',
    'passes_acc': 'first',
    'passes_inacc': 'first',
    'crosses_acc': 'first',
    'crosses_inacc': 'first',
    'lballs_acc': 'first',
    'lballs_inacc': 'first',
    'grduels_w': 'first',
    'grduels_l': 'first',
    'aerials_w': 'first',
    'aerials_l': 'first',
    'poss_lost': 'first',
    'fouls': 'first',
    'wasfouled': 'first',
    'clearances': 'first',
    'stop_shots': 'first',
    'interceptions': 'first',
    'tackles': 'first',
    'dribbled_past': 'first',
    'tballs_acc': 'first',
    'tballs_inacc': 'first',
    'ycards': 'first',
    'rcards': 'first',
    'dangmistakes': 'first',
    'countattack': 'first',
    'offsides': 'first',
    'goals_ag_otb': 'first',
    'goals_ag_itb': 'first',
    'saves_itb': 'first',
    'saves_otb': 'first',
    'saved_pen': 'first',
    'missed_penalties': 'first',
    'owngoals': 'first',
    'degree_centrality': 'first',
    'betweenness_centrality': 'first',
    'closeness_centrality': 'first',
    'flow_centrality': 'first',
    'flow_success': 'first',
    'betweenness2goals': 'first',
    'win': 'first',
    'lost': 'first',
    'is_home_team': 'first',
    'minutesPlayed': 'first',
    'game_duration': 'first'
})


In [5]:
tools.display_dataframe_to_user(name="Grouped Player Ratings Per Match", dataframe=df_grouped)

Grouped Player Ratings Per Match


In [6]:
df_dummies = pd.get_dummies(df_grouped, columns=['pos', 'pos_role'], drop_first=True)
dummy_columns = [col for col in df_dummies.columns if col.startswith('pos_') or col.startswith('pos_role_')]
df_dummies[dummy_columns] = df_dummies[dummy_columns].astype(int)
tools.display_dataframe_to_user(name="Dataset with Dummy Variables", dataframe=df_dummies)

Dataset with Dummy Variables


# II. Reg

In [7]:
X = df_dummies.drop(columns=['original_rating', 'player', 'match', 'team'])  # Drop non-numeric columns
y = df_dummies['original_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

metrics = pd.DataFrame({
    "Metric": ["Mean Absolute Error (MAE)", "Mean Squared Error (MSE)", "R-squared (R2)"],
    "Value": [mae, mse, r2]
})

tools.display_dataframe_to_user(name="Linear Regression Model Evaluation", dataframe=metrics)

Linear Regression Model Evaluation


# III. RF

In [8]:
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_grid = GridSearchCV(RandomForestRegressor(random_state=42), param_grid=rf_param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
rf_grid.fit(X_train_scaled, y_train)
best_rf = rf_grid.best_estimator_
print(best_rf)

y_pred_rf_best = best_rf.predict(X_test_scaled)

mae_rf = mean_absolute_error(y_test, y_pred_rf_best)
mse_rf = mean_squared_error(y_test, y_pred_rf_best)
r2_rf = r2_score(y_test, y_pred_rf_best)

metrics_rf = pd.DataFrame({
    "Metric": ["Mean Absolute Error (MAE)", "Mean Squared Error (MSE)", "R-squared (R2)"],
    "Value": [mae_rf, mse_rf, r2_rf]
})

tools.display_dataframe_to_user(name="Random Forest Model Evaluation", dataframe=metrics_rf)

RandomForestRegressor(min_samples_leaf=4, min_samples_split=10,
                      n_estimators=200, random_state=42)
Random Forest Model Evaluation


# IV. xgb

In [ ]:
xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 10],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0]
}

xgb_grid = GridSearchCV(XGBRegressor(random_state=42), param_grid=xgb_param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
xgb_grid.fit(X_train_scaled, y_train)
best_xgb = xgb_grid.best_estimator_
print(best_xgb)

y_pred_xgb_best = best_xgb.predict(X_test_scaled)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb_best)
mse_xgb = mean_squared_error(y_test, y_pred_xgb_best)
r2_xgb = r2_score(y_test, y_pred_xgb_best)

metrics_xgb = pd.DataFrame({
    "Metric": ["Mean Absolute Error (MAE)", "Mean Squared Error (MSE)", "R-squared (R2)"],
    "Value": [mae_xgb, mse_xgb, r2_xgb]
})

tools.display_dataframe_to_user(name="XGBoost Model Evaluation", dataframe=metrics_xgb)

# V. MLP

In [ ]:
mlp_param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [500, 1000]
}

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp_grid = GridSearchCV(MLPRegressor(random_state=42), param_grid=mlp_param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
mlp_grid.fit(X_train_scaled, y_train)
best_mlp = mlp_grid.best_estimator_

y_pred_mlp_best = best_mlp.predict(X_test_scaled)


mae_mlp = mean_absolute_error(y_test, y_pred_mlp_best)
mse_mlp = mean_squared_error(y_test, y_pred_mlp_best)
r2_mlp = r2_score(y_test, y_pred_mlp_best)

metrics_mlp = pd.DataFrame({
    "Metric": ["Mean Absolute Error (MAE)", "Mean Squared Error (MSE)", "R-squared (R2)"],
    "Value": [mae_mlp, mse_mlp, r2_mlp]
})

tools.display_dataframe_to_user(name="Neural Network Model Evaluation", dataframe=metrics_mlp)